In [1]:
from olinda.distillation import distill
from olinda.generic_model import GenericModel
from olinda.tuner import KerasTuner
from olinda.data import FeaturizedSmilesDM
from olinda.utils import get_workspace_path

/home/akumar/workspace/hub/ersilia-os/chemxor/scratch/tools/miniconda/envs/olinda-gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-07 16:11:36.139860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 16:11:36.380280: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-07 16:11:36.41361

## A model to distill

You need a model for the distillation process. Here we are creating a simple pytorch model for demo. A tensorflow model is also supported.

In [2]:
from typing import Any

import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F


class DemoModel(pl.LightningModule):
    """OlindaNet Zero: Slim(relatively) distillation network."""

    def __init__(
        self: "DemoModel"
    ) -> None:
        """Init."""
        super().__init__()
        self.conv1 = nn.Conv2d(32, 32, kernel_size=3, padding=0, stride=3)
        self.fc1 = nn.Linear(100, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)

        

    def forward(self: "DemoModel", x: Any) -> Any:
        """Forward function.

        Args:
            x (Any): model input

        Returns:
            Any: model output
        """
        x = self.conv1(x)
        x = F.relu(x)

        # flattening while keeping the batch axis
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x
###############

model = DemoModel()

Once you have a model. For distillation, it needs to be converted into a `GenericModel`.

In [3]:
model = GenericModel(model)

## Distillation

In [ ]:
student_model = distill(model, featurizer=None, working_dir=get_workspace_path(), tuner=KerasTuner([1,3]), featurized_smiles_dm=FeaturizedSmilesDM(get_workspace_path()))